In [1]:
import sys
import os
sys.path.append("../wanpy")

# from WanPy import *
from pythtb import *
from wpythtb import *
import models
import plot 

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# import numba
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sympy as sp 
from scipy.sparse.linalg import eigsh
from numba import njit, prange

%load_ext line_profiler
%timeit

In [2]:
delta = 1
t = 1
t2 = -0.3

n_super_cell = 5
model = models.Haldane(delta, t, t2).make_supercell([[n_super_cell, 0], [0, n_super_cell]])

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)

orbs = model.get_orb()
lat_vecs = model.get_lat() # lattice vectors

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
high_E_sites = np.arange(1, model.get_num_orbitals(), 2)

In [3]:
tf_list = np.random.choice(low_E_sites, 24, replace=False) # ["random", n_tf]
WFs = Wannier(model, [20, 20])
WFs.single_shot(tf_list)

In [5]:
%lprun -f WFs.subspace_selec WFs.subspace_selec(twfs=tf_list, verbose=True, iter_num=10)

0 Omega_I: 6.414520832874749
1 Omega_I: 6.4105570644219485
2 Omega_I: 6.407324223639055
3 Omega_I: 6.40458894116671
4 Omega_I: 6.40220951242489
5 Omega_I: 6.400093839511982
6 Omega_I: 6.398179116053587
7 Omega_I: 6.396420910937186
8 Omega_I: 6.394786859003617
9 Omega_I: 6.39325281018009


Timer unit: 1e-09 s

Total time: 7.51895 s
File: /Users/treycole/Codes/WanPy/dev notebooks/../WanPy/pythTB_wan.py
Function: subspace_selec at line 1904

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1904                                               def subspace_selec(
  1905                                                   self, outer_window="occupied", inner_window=None, twfs=None,
  1906                                                   N_wfs=None, iter_num=1000, tol=1e-5, alpha=1, verbose=False
  1907                                               ):
  1908                                                   # if we haven't done single-shot projection yet (set tilde states)
  1909         1       1000.0   1000.0      0.0          if twfs is not None:
  1910                                                       # initialize tilde states
  1911         1      54000.0  54000.0      0.0              twfs = self.get_trial_wfs(twfs)
  1912                                

In [8]:
%lprun -f WFs.find_optimal_subspace_bands WFs.find_optimal_subspace_bands(verbose=True, iter_num=10)

0 Omega_I: 6.391800375962284
1 Omega_I: 6.390415310447887
2 Omega_I: 6.389086410254639
3 Omega_I: 6.387804748090935
4 Omega_I: 6.386563127086495
5 Omega_I: 6.385355684853049
6 Omega_I: 6.384177601323951
7 Omega_I: 6.383024879909627
8 Omega_I: 6.381894181353485
9 Omega_I: 6.380782696047432


Timer unit: 1e-09 s

Total time: 5.8683 s
File: /Users/treycole/Codes/WanPy/dev notebooks/../WanPy/pythTB_wan.py
Function: find_optimal_subspace_bands at line 1700

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1700                                               def find_optimal_subspace_bands(
  1701                                                   self, N_wfs=None, inner_bands=None, outer_bands="occupied", 
  1702                                                   iter_num=100, verbose=False, tol=1e-10, alpha=1
  1703                                               ):
  1704                                                   """Finds the subspaces throughout the BZ that minimizes the gauge-independent spread. 
  1705                                           
  1706                                                   Used when the inner and outer windows correspond to bands rather than energy values. This function
  1707                                                   

In [6]:
nkx = 10
nky = 10
Nk = nkx*nky

k_mesh = gen_k_mesh(nkx, nky, flat=False, endpoint=False)
k_mesh_flat = gen_k_mesh(nkx, nky, flat=True, endpoint=False)

u_wfs_Wan = wf_array(model, [nkx, nky])
for i in range(k_mesh.shape[0]):
    for j in range(k_mesh.shape[1]):
        u_wfs_Wan.solve_on_one_point(k_mesh[i,j], [i,j])
psi_wfs_Wan = get_bloch_wfs(orbs, u_wfs_Wan, k_mesh)

In [7]:
spread, expc_rsq, expc_r_sq = spread_recip(lat_vecs, M, decomp=True)

print("After first projection")
print(rf"Spread = {spread[0]}")
print(rf"Omega_I = {spread[1]}")
print(rf"Omega_til = {spread[2]}")

After first projection
Spread = [0.22740033 0.22740033 0.22740033]
Omega_I = 0.527173883537877
Omega_til = 0.1550271029866458


In [8]:
# outer window of entangled bands is full occupied manifold
outer_states = u_wfs_Wan._wfs[..., :n_occ, :]

In [12]:
%lprun -f find_min_unitary find_min_unitary(u_tilde_wan, lat_vecs, orbs, eps=1e-3, iter_num=1000, verbose=True, tol=1e-17)

0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.59724334487224
6 Omega_til = 0.1543814689220185, Grad mag: 4.571752344646317
7 Omega_til = 0.15431133725559754, Grad mag: 4.546372986410804
8 Omega_til = 0.1542452878134296, Grad mag: 4.521106396698661
9 Omega_til = 0.15418296687108415, Grad mag: 4.495953588049222
10 Omega_til = 0.15412405839789625, Grad mag: 4.470915467347891
11 Omega_til = 0.15406827955937186, Grad mag: 4.44599284347493
12 Omega_til = 0.15401537679377503, Grad mag: 4.421186434330218
13 Omega_til = 0.15396512238656118, Grad mag: 4.396496873293553
14 Omega_til = 0.15391731147639431, Grad mag: 4.371924715173342
15 Omega_til = 0.15387175943563747, Grad mag: 4.3474704

Timer unit: 1e-09 s

Total time: 0.730418 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: find_min_unitary at line 766

Line #      Hits         Time  Per Hit   % Time  Line Contents
   766                                           def find_min_unitary(u_wfs, lat_vecs, orbs, eps=1 / 160, iter_num=10, verbose=False, tol=1e-12):
   767                                               """
   768                                               Finds the unitary that minimizing the gauge dependent part of the spread. 
   769                                           
   770                                               Args:
   771                                                   lat_vecs: Lattice vectors
   772                                                   M: Overlap matrix
   773                                                   eps: Step size for gradient descent
   774                                                   iter_num: Number of iterations
   775                   

In [16]:
%lprun -f find_optimal_subspace2 find_optimal_subspace2(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=100, verbose=True)

ValueError: 'shift' and 'axis' should be scalars or 1D sequences

In [15]:
%lprun -f find_optimal_subspace find_optimal_subspace(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=100, verbose=True)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

Timer unit: 1e-09 s

Total time: 1.09909 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: find_optimal_subspace at line 657

Line #      Hits         Time  Per Hit   % Time  Line Contents
   657                                           def find_optimal_subspace(
   658                                               lat_vecs, orbs, outer_states, inner_states, iter_num=100, verbose=False, tol=1e-17, alpha=1
   659                                           ):
   660         1      10000.0  10000.0      0.0      if isinstance(inner_states, wf_array):
   661                                                   shape = inner_states._wfs.shape  # [*nks, idx, orb]
   662                                                   inner_states = np.array(inner_states._wfs)
   663                                               else:
   664         1      13000.0  13000.0      0.0          shape = inner_states.shape  # [*nks, idx, orb]
   665                                           
   666       

In [13]:
%timeit find_min_unitary(u_tilde_wan, lat_vecs, orbs, eps=1e-3, iter_num=10, verbose=True, tol=1e-17)

0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.59724334487224
6 Omega_til = 0.1543814689220185, Grad mag: 4.571752344646317
7 Omega_til = 0.15431133725559754, Grad mag: 4.546372986410804
8 Omega_til = 0.1542452878134296, Grad mag: 4.521106396698661
9 Omega_til = 0.15418296687108415, Grad mag: 4.495953588049222
0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.5972433448722

In [15]:
%timeit find_min_unitary(u_tilde_wan, lat_vecs, orbs, eps=1e-3, iter_num=10, verbose=True, tol=1e-17)

0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.59724334487224
6 Omega_til = 0.1543814689220185, Grad mag: 4.571752344646317
7 Omega_til = 0.15431133725559754, Grad mag: 4.546372986410804
8 Omega_til = 0.1542452878134296, Grad mag: 4.521106396698661
9 Omega_til = 0.15418296687108415, Grad mag: 4.495953588049222
0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.5972433448722

In [19]:
%timeit find_optimal_subspace(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=10, verbose=True)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: